This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [8]:
# get api key from your .env file
#ASszyZxeUfAhykSbMB-R
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)
# type(API_KEY)

ASszyZxeUfAhykSbMB-R


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [9]:
# First, import the relevant modules
import requests
# import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [10]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key' : API_KEY, 'end_date' : '2017-12-31', 'start_date' : '2017-01-01'}
r = requests.get(url, params=params)
# print(r.status_code)
# print(r.url)
data = r.json()

In [11]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
# 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']

ds = list(data['dataset_data']['data'])
data_dict = {}
cl = list(data['dataset_data']['column_names'])

for day in ds:
    date = day[0]
    d = {}
    for i in range(1,len(day)):
        d[cl[i]] = day[i]
    data_dict[date] = d
    
openers = []

for key in data_dict.keys():
    if data_dict[key]['Open'] != None:
        openers.append(data_dict[key]['Open'])

print("Lowest Opening Price: " + str(min(openers)))
print("Highest Opening Price: " + str(max(openers)))
    
    

Lowest Opening Price: 34.0
Highest Opening Price: 53.11


In [12]:
high_and_low = []
for key in data_dict.keys():
    if (data_dict[key]['Open'] != None) and (data_dict[key]['Close'] != None):
        high_and_low.append((data_dict[key]['Close']) - (data_dict[key]['Open']))
largest_change = max(high_and_low, key=abs)
print("Largest Change in a day: " + str(largest_change))

Largest Change in a day: -1.6899999999999977


In [13]:
def day_conversion(m,d):
    calendar_days = [31,28,31,30,31,30,31,31,30,31,30,31]
    to_date = 0
    for i in range(m-1):
        to_date += calendar_days[i]
    to_date += d
    return to_date

# Check if date2 follows date1
def are_adjacent(date1, date2):
    day1 = date1.split("-")
    day2 = date2.split("-")
    y1,m1,d1 = int(day1[0]), int(day1[1]), int(day1[2])
    y2,m2,d2 = int(day2[0]), int(day2[1]), int(day2[2])
    c1 = day_conversion(m1,d1)
    c2 = day_conversion(m2,d2)
    return c2 == (c1+1)


keys = list(data_dict.keys())
keys.reverse()
two_days = []
for d1 in range(len(keys)-1):
    d2 = d1+1
    if are_adjacent(keys[d1],keys[d2]):
        if (data_dict[keys[d1]]['Close'] != None) and (data_dict[keys[d2]]['Close'] != None):
            two_days.append((data_dict[keys[d2]]['Close']) - (data_dict[keys[d1]]['Close']))
            
largest_two_days = max(two_days, key=abs)
print("Largest change between any two days: " + str(largest_two_days))
    

Largest change between any two days: -2.559999999999995


In [14]:
trade_vol = []
for key in keys:
    if data_dict[key]['Traded Volume'] != None:
        trade_vol.append(data_dict[key]['Traded Volume'])
avg_trade = sum(trade_vol) / len(trade_vol)
print("Average daily trading volume: " + str(avg_trade))

Average daily trading volume: 89124.33725490196


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)